# Advanced Programming Summative Assignment

Please see the README file (README.md) for a full overview of the project, installation instructions and running instructions. 

## Contents

#### 1.0 [Package installations and import statements](#1.0-Package-installations-and-import-statements)

#### 2.0 [Data extraction and cleaning](#2.0-Data-extraction-and-cleaning)

2.1 [Read CSVs to dataframes and general data cleaning](#2.2-Merge-airport-and-frequency-dataframes)

2.2 [Merge airport and frequency dataframes](#2.1-Read-CSVs-to-dataframes-and-general-data-cleaning)

2.3 [Transform dataframes to JSON](#2.3-Transform-dataframes-to-JSON)

#### 3.0 [Load data to MySQL database](#3.0-Load-data-to-MySQL-database)


### 1.0 Package installations and import statements

In [3]:
# Installing missing packages
import sys
%conda install --yes --prefix {sys.prefix} mysqlclient
%conda install --yes --prefix {sys.prefix} mysql-connector-python 
%conda install --yes --prefix {sys.prefix} numpy
%conda install --yes --prefix {sys.prefix} pyodbc
%conda install --yes --prefix {sys.prefix} pymysql
%conda install --yes --prefix {sys.prefix} sqlalchemy 
%conda install --yes --prefix {sys.prefix} pandastable


Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::anaconda==2021.05=py38_0
  - defaults/osx-64::conda-build==3.21.4=py38hecd8cb5_0
failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::anaconda==2021.05=py38_0
  - defaults/osx-64::conda-build==3.21.4=py38hecd8cb5_0
done

## Package Plan ##

  environment location: /Users/tompain/opt/anaconda3

  added / updated specs:
    - mysql-connector-python


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2020.07  |        

In [3]:
# Import packages
import os
import pandas as pd
import numpy as np
import json
import pprint
import pymysql
import mysql.connector
# import matplotlib.pyplot as plt
# import seaborn as sns
import tkinter as tk

In [12]:
username = "newuser"
password = "Eu&AMsB2pk6bKjNV71W"

#Testing MySQL connection
mydb = mysql.connector.connect(
  host="localhost",
  user=username,
  passwd=password, 
)

print(mydb)

In [ ]:
#Create database (only run once)
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE if not exists dummy_airports")
mycursor.execute("use dummy_airports")
mycursor.execute("""CREATE TABLE airports_frequencies(
airport_ref int primary key,
ident varchar(50),
type varchar(50),
name varchar(255),
latitude_deg float,
longitude_deg float,
elevation_ft int,
continent varchar(50),
iso_country varchar(50),
iso_region varchar(50),
municipality varchar(50),
scheduled_service varchar(50),
gps_code varchar(50),
iata_code varchar(50),
wikipedia_link varchar(255),
small_airport int,
medium_airport int,
large_airport int,
frequency_mhz varchar(255));""")



### 2.0 Data extraction and cleaning

A lot of the data processing is generic and not specific to the original data. This allows the application to handle other datasets from the same source. 

#### 2.1 Read CSVs to dataframes and general data cleaning

In [4]:
# Extract data from CSV files
df_runways = pd.read_csv ('data/runways.csv', index_col=['id'])
df_frequencies = pd.read_csv ('data/airport-frequencies.csv', index_col=['id'])
# Prevent pandas from replacing the 'continent' value 'NA' (for North America) with NaN 
df_airports = pd.read_csv ('data/airports.csv', keep_default_na=False)

# Remove any rows with all data missing
df_airports.dropna(how='all')
df_runways.dropna(how='all')
df_frequencies.dropna(how='all')

# Remove any duplicated rows
df_airports.drop_duplicates()
df_runways.drop_duplicates()
df_frequencies.drop_duplicates()

# Remove unneeded columns
df_airports.drop(['keywords','home_link','local_code'], axis='columns', inplace=True)
df_runways.drop(['airport_ident'], axis='columns', inplace=True)
df_frequencies.drop(['airport_ident','description','type'], axis='columns', inplace=True)

# change the airport column id name to airport_ref to align with other data
df_airports.rename(columns={"id": "airport_ref"}, inplace=True)

# Remove rows that do not have a valid airport_ref
df_airports = df_airports[df_airports['airport_ref'].apply(lambda x: str(x).isdigit())]
df_runways = df_runways[df_runways['airport_ref'].apply(lambda x: str(x).isdigit())]
df_frequencies = df_frequencies[df_frequencies['airport_ref'].apply(lambda x: str(x).isdigit())]

# Add columns to the Airports df for small, medium and large airports with binary values
df_airports['small_airport'] = df_airports.type == 'small_airport'
df_airports['medium_airport'] = df_airports.type == 'medium_airport'
df_airports['large_airport'] = df_airports.type == 'large_airport'
df_airports['small_airport'] = df_airports['small_airport'].astype(int) 
df_airports['medium_airport'] = df_airports['medium_airport'].astype(int) 
df_airports['large_airport'] = df_airports['large_airport'].astype(int) 

# filter out closed airports - may need to force a copy, not sure yet - may need to do this after frequencies are added
df_airports = df_airports[(df_airports.type != 'closed')]

#### 2.2 Merge airport and frequency dataframes

In [5]:
# Create an empty dict to hold one key for each airport with a nested list of frequencies 
frequencies_dict = {}

# Iterate through the frequencies creating one key for each airport with a list for it's frequencies 
for index, row in df_frequencies.iterrows():
    if row['airport_ref'] not in frequencies_dict:
        frequencies_dict[row['airport_ref']] = [row['frequency_mhz']]
    else:
        frequencies_dict[row['airport_ref']].append(row['frequency_mhz'])

# Create a pandas series of airports (as index) and frequency lists 
df_frequencies_series = pd.Series(frequencies_dict, name='df_frequencies_series')
# Rename the column titles to align with other data 
df_airports_frequencies = df_frequencies_series.to_frame()
df_airports_frequencies.index.name = 'airport_ref'
# Convert to dataframe and merge with airports dataframe 
df_airports_frequencies.rename(columns={'df_frequencies_series': 'frequency_mhz'}, inplace = True)
df_airports_frequencies = pd.merge(df_airports, df_airports_frequencies, on="airport_ref", how = 'left')

#### 2.3 Transform dataframes to JSON

In [9]:
# write combined airport and frequency data to JSON
airports_frequencies_json = df_airports_frequencies.to_json(orient = 'records')
airports_frequencies_json_list = json.loads(airports_frequencies_json)
         
# write runways data to JSON
runways_json = df_runways.to_json(orient = 'records')
runways_json_list = json.loads(runways_json)

# pprint.pprint(airports_frequencies_json_list[1:5])
df_airports_frequencies.to_csv('airport-frequencies.csv')
df_runways.to_csv('runways.csv')

### 3.0 Load data to MySQL database